In [16]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key
import os

In [17]:
# open csv from part 1
city_data_df = pd.read_csv(os.path.join("weather_data","WeatherPy_challenge.csv"))
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Description,Rain,Snow,Date
0,0,Qaanaaq,77.48,-69.36,-14.51,58,0,5.61,GL,clear sky,0.00,0.0,2020-02-20 22:46:30
1,1,Hobart,-42.88,147.33,61.00,44,40,13.87,AU,light rain,0.25,0.0,2020-02-20 22:46:30
2,2,Albany,42.60,-73.97,26.60,38,1,16.11,US,clear sky,0.00,0.0,2020-02-20 22:45:03
3,3,Port Alfred,-33.59,26.89,68.00,92,59,6.04,ZA,broken clouds,0.00,0.0,2020-02-20 22:46:31
4,4,Najran,17.49,44.13,73.40,35,53,4.70,SA,broken clouds,0.00,0.0,2020-02-20 22:46:32


In [19]:
#create separate df's from the csv
paloalto_city_df = city_data_df.loc[city_data_df["City"] == "Palo Alto"]
hillsborough_city_df = city_data_df.loc[city_data_df["City"] == "Hillsborough"]
halfmoon_city_df = city_data_df.loc[city_data_df["City"] == "Half Moon Bay"]
pacificgrove_city_df = city_data_df.loc[city_data_df["City"] == "Pacific Grove"]
selma_city_df = city_data_df.loc[city_data_df["City"] == "Selma"]

In [26]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

## Map Figure with Directions Layer

In [70]:
# Latitude-longitude pairs
paloalto_lat_long = tuple([x for x in paloalto_city_df[["Lat","Lng"]].to_records(index=False)][0])
hillsborough_lat_long = tuple([x for x in hillsborough_city_df[["Lat","Lng"]].to_records(index=False)][0])
halfmoon_lat_long = tuple([x for x in halfmoon_city_df[["Lat","Lng"]].to_records(index=False)][0])
pacificgrove_lat_long = tuple([x for x in pacificgrove_city_df[["Lat","Lng"]].to_records(index=False)][0])
selma_lat_long = tuple([x for x in selma_city_df[["Lat","Lng"]].to_records(index=False)][0])

# Create the figure with directions layer
fig = gmaps.figure()
norcal_directions_layer = gmaps.directions_layer(paloalto_lat_long,
                                                 selma_lat_long,
                                                 waypoints=[hillsborough_lat_long,halfmoon_lat_long,pacificgrove_lat_long],
                                                 travel_mode="DRIVING")
fig.add_layer(norcal_directions_layer)
fig

Figure(layout=FigureLayout(height='420px'))

## Marker Layer Map

In [83]:
#Append the df's to 1 df ignoring the original index
norcal_city_df = paloalto_city_df.append(hillsborough_city_df, ignore_index = True) \
                                        .append(halfmoon_city_df, ignore_index = True) \
                                        .append(pacificgrove_city_df, ignore_index = True) \
                                        .append(selma_city_df, ignore_index = True)
# make room for the hotel name
norcal_city_df["Hotel Name"] = ""
norcal_city_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Description,Rain,Snow,Date,Hotel Name
0,350,Palo Alto,37.44,-122.14,71.01,37,1,5.82,US,clear sky,0.0,0.0,2020-02-20 22:49:22,
1,289,Hillsborough,37.57,-122.38,71.01,32,1,5.82,US,clear sky,0.0,0.0,2020-02-20 22:49:10,
2,257,Half Moon Bay,37.46,-122.43,71.01,51,75,3.36,US,broken clouds,0.0,0.0,2020-02-20 22:48:52,
3,68,Pacific Grove,36.62,-121.92,72.00,36,1,4.70,US,clear sky,0.0,0.0,2020-02-20 22:47:07,
4,275,Selma,36.57,-119.61,75.99,24,1,1.23,US,clear sky,0.0,0.0,2020-02-20 22:49:02,


In [84]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in norcal_city_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        norcal_city_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

In [85]:
#Df with hotel name populated
norcal_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Description,Rain,Snow,Date,Hotel Name
0,350,Palo Alto,37.44,-122.14,71.01,37,1,5.82,US,clear sky,0.0,0.0,2020-02-20 22:49:22,Stanford Terrace Inn
1,289,Hillsborough,37.57,-122.38,71.01,32,1,5.82,US,clear sky,0.0,0.0,2020-02-20 22:49:10,Hyatt Regency San Francisco Airport
2,257,Half Moon Bay,37.46,-122.43,71.01,51,75,3.36,US,broken clouds,0.0,0.0,2020-02-20 22:48:52,"The Ritz-Carlton, Half Moon Bay"
3,68,Pacific Grove,36.62,-121.92,72.00,36,1,4.70,US,clear sky,0.0,0.0,2020-02-20 22:47:07,Lovers Point Inn
4,275,Selma,36.57,-119.61,75.99,24,1,1.23,US,clear sky,0.0,0.0,2020-02-20 22:49:02,Holiday Inn Selma-Swancourt


In [86]:
#setup for the marker
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>Max Temp: {Max Temp} °F, {Description}</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in norcal_city_df.iterrows()]

In [88]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = norcal_city_df[["Lat", "Lng"]]
#max_temp = norcal_city_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
#heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
#             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))